# Structural File Analysis
This notebook contains ways to visualize the static state of a folder of source files and its subdirectories. This does not do any analysis of the contents of these files or any historical analysis.

For best results, mark this notebook as trusted so HTML content on graphs can execute.

In [1]:
# Install statements for things not present in Anaconda
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [85]:
# Standard Imports and path.

import pandas as pd
import plotly.express as px

from FileUtilities import get_source_file_metrics

# Path should point to the root directory of the application you're analyzing
path = 'M:/dev/CodeVis/OpenRA/'
print('Working with files at ' + path)

Working with files at M:/dev/CodeVis/OpenRA/


In [84]:
# Grab the file metrics for source files and put them into a data frame
df = pd.DataFrame(get_source_file_metrics(path))

# Write to a file for other analyses processes
df.to_csv('filesizes.csv')

# Sort AFTER we export
df = df.sort_values('lines', ascending=False)

# Display top 5 for preview purposes
df.head()

,fullpath,project,path,filename,ext,lines
159,OpenRA.Game/Server/Server.cs,OpenRA.Game,Server,Server.cs,.cs,1400
83,OpenRA.Game/Map/Map.cs,OpenRA.Game,Map,Map.cs,.cs,1379
584,OpenRA.Mods.Common/Traits/Air/Aircraft.cs,OpenRA.Mods.Common,Traits/Air,Aircraft.cs,.cs,1316
567,OpenRA.Mods.Common/ServerTraits/LobbyCommands.cs,OpenRA.Mods.Common,ServerTraits,LobbyCommands.cs,.cs,1149
759,OpenRA.Mods.Common/Traits/Mobile.cs,OpenRA.Mods.Common,Traits,Mobile.cs,.cs,1032


In [5]:
# Statistical Analysis
df.describe()

,lines
count,1358.000000
mean,137.070692
std,146.203261
min,18.000000
25%,56.000000
50%,88.000000
75%,160.000000
max,1400.000000


In [83]:
# Histogram of lines of code
fig = px.histogram(df,
                   x="lines",
                   title='Frequency of File Sizes',
                   labels={'lines': 'Lines of Code'},
                   color_discrete_sequence=px.colors.qualitative.Dark24)
fig.show()

In [97]:
# Overall box plot for all code
fig = px.box(df,
             title='Distribution of Lines of Code',
             x='lines',
             height=275,
             hover_data=['project','path','filename'],
             labels={
                 'project': 'Project',
                 'lines': 'Lines of Code',
             },
             points='suspectedoutliers')
fig.update_traces(quartilemethod='linear', jitter=1)
fig.show()

In [98]:
# Box plot for distribution by projects
fig = px.box(df,
             x='lines',
             y='project',
             hover_data=['path','filename'],
             labels={
               'project': 'Project',
               'lines': 'Lines of Code',
             },
             title='Distribution of Lines of Code by Project',
             points='suspectedoutliers',
             color_discrete_sequence=px.colors.qualitative.Dark24,
             color='project')
fig.update_traces(quartilemethod='linear', jitter=0.5)
fig.update_layout(showlegend=False)
fig.update_yaxes(tickangle=30, tickfont={'size': 10})
fig.show()

In [81]:
# Cool color scales: matter, icefire, darkmint, picnic, temps, balance, tempo

# TODO: Using Graph Objects here would allow for a more dynamic hierarchy

fig = px.treemap(df,
                 path=[px.Constant('All Code'),'project','path','filename'],
                 color='lines',
                 title='Size of Code Files by Project and Directory',
                 hover_name='filename',
                 hover_data=['fullpath'],
                 color_continuous_scale='balance',
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [82]:
# Sunburst diagram. Same data as a treemap, but different presentation
fig = px.sunburst(df,
                 path=['project','path','filename'],
                 color='lines',
                 title='Size of Code Files by Project and Directory',
                 hover_data=['fullpath'],
                 color_continuous_scale='sunsetdark',
                 width=1024,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()